In [1]:
import Importer  # Help directly import Ipython notebooks
# (Reference: https://nbviewer.jupyter.org/github/jupyter/notebook/blob/master/docs/source/examples/Notebook/Importing%20Notebooks.ipynb)
import Tushare_data_fetcher
import pymysql
import tushare as ts
import numpy as np
import pandas as pd
import sqlalchemy
import statsmodels.formula.api
import scipy
import datetime
import json

importing Jupyter notebook from Tushare_data_fetcher.ipynb


In [2]:
class DataLoader:
    def __init__(self):
        # pymysql
        with open('../config/config.json', 'r') as f:
            config = json.load(f)
        con = pymysql.connect(**config)
        cur = con.cursor()

        # sqlalchemy
        alchemy_str = 'mysql+pymysql://' + config['user'] + ':' + config[
            'password'] + '@' + config['host'] + ':' + str(
                config['port']) + '/' + config['database']
        self.alchemy_engine = sqlalchemy.create_engine(alchemy_str)

        # Get calendar dataframe for convenient date computation
        cal_query = '''
            SELECT DISTINCT trade_date
            FROM asset_return;
            '''
        cal_df = pd.read_sql_query(cal_query, con=self.alchemy_engine)
        cal_df = cal_df.sort_values(by=['trade_date'], ascending=False)
        self.cal_df = cal_df
        self.cal_df.reset_index(drop=True, inplace=True)
        # Str version for indexing
        cal_str_df = pd.DataFrame()
        cal_str_df['trade_date_str'] = cal_df['trade_date'].apply(
            lambda x: x.strftime('%Y-%m-%d'))
        self.cal_str_df = cal_str_df
        self.cal_str_df.reset_index(drop=True, inplace=True)
        # Store the latest date
        self.latest_date = self._get_latest_date()
        return

    def _get_latest_date(self, lag=0):  # Get latest date in database BARRA
        date_dt = self.cal_df.iloc[lag, 0]
        date = date_dt.strftime('%Y-%m-%d')
        print(f'Lastest date(lag = {lag} from today): {date}')
        return date

    def _get_query_end_date(self, lag=0):
        # Get an end date for query, so that record numbers fit the expectations
        if (lag == 0):
            dt = datetime.datetime.now()
            end_date = dt.strftime(
                '%Y-%m-%d'
            )  # Today would be a natural end date since it could never be accessed
        else:
            end_date = self._get_latest_date(lag=lag - 1)
        return end_date

    def _last_disclosure_date(self, nominal_end_date_dt):
        # Find an approriate query_date for balance query
        # Last disclosure date for 1Q 4.30
        # Last disclosure date for 1H 8.31
        # Last disclosure date for 3Q 10.31
        # Last disclosure date for 1Y 4.30(Next year)
        nominal_end_date = nominal_end_date_dt.strftime('%Y-%m-%d')
        nominal_end_year = nominal_end_date[:4]
        if (nominal_end_date.endswith('03-31')):
            last_disclosure_date = nominal_end_year + '-04-30'
        elif (nominal_end_date.endswith('06-30')):
            last_disclosure_date = nominal_end_year + '-08-31'
        elif (nominal_end_date.endswith('09-30')):
            last_disclosure_date = nominal_end_year + '-10-31'
        else:
            last_disclosure_date = str(int(nominal_end_year)+1) + '-04-30'
        return last_disclosure_date

    def _get_date_lag(self, date):
        lag = self.cal_str_df[self.cal_str_df['trade_date_str'] ==
                              date].index.to_list()[0]
        return lag

    def _standardize_date(self, date_str):  # Standardize for query
        date_dt = datetime.datetime.strptime(date_str, '%Y%m%d')
        standard_str = date_dt.strftime('%Y-%m-%d')
        return standard_str

    def _winsorize(self, data, limits=[0.01, 0.01]):
        data_winsorize = scipy.stats.mstats.winsorize(data, limits=limits)
        data_winsorize = data_winsorize.data
        return data_winsorize

    def _standardize(self, data):
        mu = np.mean(data)
        sigma = np.std(data)
        data_standardize = (data - mu) / sigma
        return data_standardize

    def get_daily_size(self, date=None):
        if date == None:  # Default: Get latest date
            date = self.latest_date
        else:
            date = self._standardize_date(date)
        query = f'''
            SELECT ts_code, total_mv
            FROM daily
            WHERE trade_date = \'{date}\';
            '''
        query_df = pd.read_sql_query(query, con=self.alchemy_engine)
        query_df['total_mv_exact'] = query_df['total_mv'] * 10000
        query_df['log_mv'] = np.log(query_df['total_mv_exact'])
        return_df = query_df[['ts_code', 'log_mv']].copy()
        return return_df

    def get_daily_book_to_price(self, date=None):
        if date == None:  # Default: Get latest date
            date = self.latest_date
        else:
            date = self._standardize_date(date)

        query = f'''
            SELECT ts_code, pb
            FROM daily
            WHERE trade_date = \'{date}\';
            '''
        query_df = pd.read_sql_query(query, con=self.alchemy_engine)
        # PB versus Book-to-Price
        query_df['book_to_price'] = query_df['pb'].apply(lambda x: 1 / x)
        return_df = query_df[['ts_code', 'book_to_price']].copy()
        return return_df

    def get_daily_nonlinear_size(self, date=None):
        if date == None:  # Default: Get latest date
            date = self.latest_date
        else:
            date = self._standardize_date(date)

        query = f'''
            SELECT ts_code, total_mv
            FROM daily
            WHERE trade_date = \'{date}\';
            '''
        query_df = pd.read_sql_query(query, con=self.alchemy_engine)
        query_df['total_mv_cube'] = query_df['total_mv'].apply(lambda x: x**3)
        # Linear regression
        size_model = statsmodels.formula.api.ols(
            formula='total_mv_cube ~ total_mv', data=query_df).fit()
        size_resid = size_model.resid
        # Winsorize and standardize
        size_resid_winsorize = self._winsorize(size_resid)
        size_resid_standardize = self._standardize(size_resid_winsorize)

        query_df['nonlinear_size'] = size_resid_standardize
        return_df = query_df[['ts_code', 'nonlinear_size']].copy()
        return return_df

    def get_daily_beta(self, date=None):
        if date == None:  # Default: Get latest date
            date = self.latest_date
        else:
            date = self._standardize_date(date)

        date_lag = self._get_date_lag(date)
        latest_date_252 = self._get_latest_date(lag=date_lag + 252)
        query_end_date = self._get_query_end_date(lag=date_lag)

        # Get all stock return from SQL into memory(read data from SQL is time-consuming)
        query_all_stock = f'''
            SELECT ts_code, pct_chg, trade_date
            FROM asset_return
            WHERE trade_date > \'{latest_date_252}\'
            AND trade_date < \'{query_end_date}\'
            AND ts_code <> '000985.CSI';
            '''
        all_stock_df = pd.read_sql_query(query_all_stock,
                                         con=self.alchemy_engine)
        # Get all stock_id
        all_stock_id_list = list(set(all_stock_df['ts_code'].to_list()))
        # Get risk-free return (shibor_on) during this period
        shibor_query = f'''
            SELECT `on`, date
            FROM shibor
            WHERE date > \'{latest_date_252}\'
            AND date < \'{query_end_date}\';
            '''
        shibor_df = pd.read_sql_query(shibor_query, con=self.alchemy_engine)
        shibor_df['daily_on'] = shibor_df['on'].apply(lambda x: (np.power(
            (1 + x / 100), 1 / 252) - 1) * 100)
        # Get market portfolio return (000985.CSI, alias mkp) during this period
        query_index = f'''
            SELECT pct_chg, trade_date
            FROM asset_return
            WHERE ts_code = '000985.CSI'
            AND trade_date > \'{latest_date_252}\'
            AND trade_date < \'{query_end_date}\';
            '''
        mkp_df = pd.read_sql_query(query_index, con=self.alchemy_engine)
        # Define a container for beta
        beta_container = []
        count = 1
        for stock_id in all_stock_id_list:
            stock_df = all_stock_df[all_stock_df['ts_code'] == stock_id].copy()
            ret_df = mkp_df.merge(right=stock_df,
                                  how='inner',
                                  on='trade_date',
                                  suffixes=['_stock', '_mkp'])
            if (ret_df.shape[0] < 126):  # Not enough records for computation
                print(
                    f'({count}/{len(all_stock_id_list)}) {stock_id} has inadequate records till {date}.'
                )
                count += 1
                beta_container.append([stock_id, np.nan])
                pass
            else:
                ret_df = ret_df.merge(right=shibor_df,
                                      how='inner',
                                      left_on='trade_date',
                                      right_on='date')
                ret_df = ret_df.sort_values(by=['trade_date'], ascending=False)
                # Weigh the excess return with EWMA weights (half-life = 63)
                half_life = 63
                alpha = np.power(0.5, 1 / half_life)
                weight = np.logspace(start=1, stop=252, num=252, base=alpha)
                record_num = ret_df.shape[0]
                weight = weight[:record_num]
                # Store weighed excess return series for regression
                reg_df = pd.DataFrame()
                reg_df['pct_chg_stock_excess'] = (ret_df['pct_chg_stock'] -
                                                  ret_df['daily_on'])
                reg_df['pct_chg_mkp_excess'] = ret_df['pct_chg_mkp'] - ret_df[
                    'daily_on']
                beta_model = statsmodels.formula.api.wls(
                    formula='pct_chg_stock_excess ~ pct_chg_mkp_excess',
                    data=reg_df,
                    weights=weight).fit()
                # Store the regression coefficients
                beta = beta_model.params['pct_chg_mkp_excess']
                print(
                    f'({count}/{len(all_stock_id_list)}) Adjusted beta of {stock_id} in {date} is {beta}.'
                )
                count += 1
                beta_container.append([stock_id, beta])
        return_df = pd.DataFrame(beta_container,
                                 columns=['ts_code', 'beta_raw'])
        return return_df

    def get_daily_momentum(self, date=None):
        if date == None:  # Default: Get latest date
            date = self.latest_date
        else:
            date = self._standardize_date(date)
        date_lag = self._get_date_lag(date)
        latest_date_526 = self._get_latest_date(lag=date_lag +
                                                526)  # 504 + 21 + 1
        latest_date_21 = self._get_latest_date(lag=date_lag + 21)
        query_all_stock = f'''
                    SELECT ts_code, pct_chg, trade_date
                    FROM asset_return
                    WHERE trade_date > \'{latest_date_526}\'
                    AND trade_date <= \'{latest_date_21}\'
                    AND ts_code <> '000985.CSI';
                    '''
        all_stock_df = pd.read_sql_query(query_all_stock,
                                         con=self.alchemy_engine)
        # Get all stock_id
        all_stock_id_list = list(set(all_stock_df['ts_code'].to_list()))
        # Get risk-free return (shibor_on) during this period
        shibor_query = f'''
            SELECT `on`, date
            FROM shibor
            WHERE date > \'{latest_date_526}\'
            AND date <= \'{latest_date_21}\';
            '''
        shibor_df = pd.read_sql_query(shibor_query, con=self.alchemy_engine)
        shibor_df['daily_on'] = shibor_df['on'].apply(lambda x: np.power(
            (1 + x / 100), 1 / 252) - 1)
        # Note that this 'daily_on' is computed without pct adjustment, considering its use in the formula
        momentum_container = []
        count = 1
        suc_count = 1  # Count stocks with enough records
        for stock_id in all_stock_id_list:
            stock_df = all_stock_df[all_stock_df['ts_code'] == stock_id].copy()
            stock_df['chg'] = stock_df['pct_chg'] / 100
            if (
                    stock_df.shape[0] < 500
            ):  # Inadequate records (Spare five days to get more useful data)
                momentum_container.append([stock_id, np.nan,
                                           0])  # No records are used
                print(
                    f'({count}/{len(all_stock_id_list)}) {stock_id} has inadequate records (only {stock_df.shape[0]}) till {date}.'
                )
                count += 1
                pass
            else:
                ret_df = stock_df.merge(right=shibor_df,
                                        how='inner',
                                        left_on='trade_date',
                                        right_on='date')
                ret_df = ret_df.sort_values(by=['trade_date'], ascending=False)
                ret_df['daily_RSTR'] = np.log(1 + ret_df['chg']) - np.log(
                    1 + ret_df['daily_on'])
                # Weigh the excess return with EWMA weights (half-life = 63)
                half_life = 126
                alpha = np.power(0.5, 1 / half_life)
                valid_days = ret_df.shape[0]
                weight = np.logspace(start=1,
                                     stop=valid_days,
                                     num=valid_days,
                                     base=alpha)
                # Compute RSTR
                RSTR = np.sum(ret_df['daily_RSTR'] * weight)
                momentum_container.append([stock_id, RSTR, valid_days])
                print(
                    f'({count}/{len(all_stock_id_list)}) RSTR of {stock_id} in {date} is {RSTR}. Current Success: {suc_count}'
                )
                count += 1
                suc_count += 1
        return_df = pd.DataFrame(
            momentum_container,
            columns=['ts_code', 'momentum_raw', 'valid_days_used'])
        return return_df

    def _get_daily_RV_HSIGMA(self, date=None):
        if date == None:  # Default: Get latest date
            date = self.latest_date
        else:
            date = self._standardize_date(date)

        date_lag = self._get_date_lag(date)
        latest_date_252 = self._get_latest_date(lag=date_lag + 252)
        query_end_date = self._get_query_end_date(lag=date_lag)
        # Get all stock return from SQL into memory(read data from SQL is time-consuming)
        query_all_stock = f'''
            SELECT ts_code, pct_chg, trade_date
            FROM asset_return
            WHERE trade_date > \'{latest_date_252}\'
            AND trade_date < \'{query_end_date}\'
            AND ts_code <> '000985.CSI';
            '''
        all_stock_df = pd.read_sql_query(query_all_stock,
                                         con=self.alchemy_engine)
        # Get all stock_id
        all_stock_id_list = list(set(all_stock_df['ts_code'].to_list()))
        # Get risk-free return (shibor_on) during this period
        shibor_query = f'''
            SELECT `on`, date
            FROM shibor
            WHERE date > \'{latest_date_252}\'
            AND date < \'{query_end_date}\';
            '''
        shibor_df = pd.read_sql_query(shibor_query, con=self.alchemy_engine)
        shibor_df['daily_on'] = shibor_df['on'].apply(lambda x: (np.power(
            (1 + x / 100), 1 / 252) - 1) * 100)
        # Get market portfolio return (000985.CSI, alias mkp) during this period
        query_index = f'''
            SELECT pct_chg, trade_date
            FROM asset_return
            WHERE ts_code = '000985.CSI'
            AND trade_date > \'{latest_date_252}\'
            AND trade_date < \'{query_end_date}\';
            '''
        mkp_df = pd.read_sql_query(query_index, con=self.alchemy_engine)
        # Define a container for beta
        HSIGMA_container = []  # Historical Sigma
        count = 1
        for stock_id in all_stock_id_list:
            stock_df = all_stock_df[all_stock_df['ts_code'] == stock_id].copy()
            ret_df = mkp_df.merge(right=stock_df,
                                  how='inner',
                                  on='trade_date',
                                  suffixes=['_stock', '_mkp'])
            if (ret_df.shape[0] < 126):  # Not enough records for computation
                print(
                    f'({count}/{len(all_stock_id_list)}) {stock_id} has inadequate records till {date}.'
                )
                count += 1
                HSIGMA_container.append([stock_id, np.nan])
                pass
            else:
                ret_df = ret_df.merge(right=shibor_df,
                                      how='inner',
                                      left_on='trade_date',
                                      right_on='date')
                ret_df = ret_df.sort_values(by=['trade_date'], ascending=False)
                # Weigh the excess return with EWMA weights (half-life = 63)
                half_life = 63
                alpha = np.power(0.5, 1 / half_life)
                weight = np.logspace(start=1, stop=252, num=252, base=alpha)
                record_num = ret_df.shape[0]
                weight = weight[:record_num]
                # Store weighed excess return series for regression
                reg_df = pd.DataFrame()
                reg_df['pct_chg_stock_excess'] = (ret_df['pct_chg_stock'] -
                                                  ret_df['daily_on'])
                reg_df['pct_chg_mkp_excess'] = ret_df['pct_chg_mkp'] - ret_df[
                    'daily_on']
                beta_model = statsmodels.formula.api.wls(
                    formula='pct_chg_stock_excess ~ pct_chg_mkp_excess',
                    data=reg_df,
                    weights=weight).fit()
                # Store the regression coefficients
                resid_ret = beta_model.resid
                resid_ret_sigma = np.std(resid_ret)
                print(
                    f'({count}/{len(all_stock_id_list)}) Volatility of residual return of {stock_id} in {date} is {resid_ret_sigma}.'
                )
                count += 1
                HSIGMA_container.append([stock_id, resid_ret_sigma])
        return_df = pd.DataFrame(HSIGMA_container,
                                 columns=['ts_code', 'HSIGMA_raw'])
        return return_df

    def _get_daily_RV_DASTD(self, date=None):
        if date == None:  # Default: Get latest date
            date = self.latest_date
        else:
            date = self._standardize_date(date)

        date_lag = self._get_date_lag(date)
        latest_date_252 = self._get_latest_date(lag=date_lag + 252)
        query_end_date = self._get_query_end_date(lag=date_lag)
        # Get all stock return from SQL into memory(read data from SQL is time-consuming)
        query_all_stock = f'''
            SELECT ts_code, pct_chg, trade_date
            FROM asset_return
            WHERE trade_date > \'{latest_date_252}\'
            AND trade_date < \'{query_end_date}\'
            AND ts_code <> '000985.CSI';
            '''
        all_stock_df = pd.read_sql_query(query_all_stock,
                                         con=self.alchemy_engine)
        # Get all stock_id
        all_stock_id_list = list(set(all_stock_df['ts_code'].to_list()))
        # Get risk-free return (shibor_on) during this period
        shibor_query = f'''
            SELECT `on`, date
            FROM shibor
            WHERE date > \'{latest_date_252}\'
            AND date < \'{query_end_date}\';
            '''
        shibor_df = pd.read_sql_query(shibor_query, con=self.alchemy_engine)
        shibor_df['daily_on'] = shibor_df['on'].apply(lambda x: (np.power(
            (1 + x / 100), 1 / 252) - 1) * 100)
        # Define a container
        DASTD_container = []
        count = 1
        for stock_id in all_stock_id_list:
            stock_df = all_stock_df[all_stock_df['ts_code'] == stock_id].copy()
            if (stock_df.shape[0] < 126):  # Not enough records for computation
                print(
                    f'({count}/{len(all_stock_id_list)}) {stock_id} has inadequate records till {date}.'
                )
                count += 1
                DASTD_container.append([stock_id, np.nan, np.nan])
                pass
            else:
                ret_df = stock_df.merge(right=shibor_df,
                                        how='inner',
                                        left_on='trade_date',
                                        right_on='date')
                ret_df = ret_df.sort_values(by=['trade_date'], ascending=False)
                # Weigh the excess return with EWMA weights (half-life = 63)
                half_life = 42
                alpha = np.power(0.5, 1 / half_life)
                weight = np.logspace(start=1, stop=252, num=252, base=alpha)
                record_num = ret_df.shape[0]
                weight = weight[:record_num]
                # Compute wighted standard deviation
                ret_df['daily_excessive_ret'] = (ret_df['pct_chg'] -
                                                 ret_df['daily_on']) / 100
                # Use the real return, rather than the pct form
                ret_df['ex_ret_temp'] = np.square(
                    ret_df['daily_excessive_ret'] -
                    np.mean(ret_df['daily_excessive_ret']))
                DASTD = np.mean(weight * ret_df['ex_ret_temp'])
                DASTD_2 = np.std(ret_df['daily_excessive_ret'] * weight)
                print(
                    f'({count}/{len(all_stock_id_list)}) Volatility of daily excessive return of {stock_id} in {date} is {DASTD}.'
                )
                count += 1
                DASTD_container.append([stock_id, DASTD, DASTD_2])
        return_df = pd.DataFrame(
            DASTD_container,
            columns=['ts_code', 'DASTD_raw', 'DASTD_weighed_ret_raw'])
        return return_df

    def _get_daily_RV_CMRA(self, date=None):
        if date == None:  # Default: Get latest date
            date = self.latest_date
        else:
            date = self._standardize_date(date)

        date_lag = self._get_date_lag(date)
        latest_date_252 = self._get_latest_date(lag=date_lag + 252)
        query_end_date = self._get_query_end_date(lag=date_lag)
        # Get all stock return from SQL into memory(read data from SQL is time-consuming)
        query_all_stock = f'''
            SELECT ts_code, pct_chg, trade_date
            FROM asset_return
            WHERE trade_date > \'{latest_date_252}\'
            AND trade_date < \'{query_end_date}\'
            AND ts_code <> '000985.CSI';
            '''
        all_stock_df = pd.read_sql_query(query_all_stock,
                                         con=self.alchemy_engine)
        # Get all stock_id
        all_stock_id_list = list(set(all_stock_df['ts_code'].to_list()))
        # Get risk-free return (shibor_on) during this period
        shibor_query = f'''
            SELECT `on`, date
            FROM shibor
            WHERE date > \'{latest_date_252}\'
            AND date < \'{query_end_date}\';
            '''
        shibor_df = pd.read_sql_query(shibor_query, con=self.alchemy_engine)
        shibor_df['daily_on'] = shibor_df['on'].apply(lambda x: (np.power(
            (1 + x / 100), 1 / 252) - 1))  # Not use the pct form
        # Define a container
        CMRA_container = []
        count = 1
        for stock_id in all_stock_id_list:
            stock_df = all_stock_df[all_stock_df['ts_code'] == stock_id].copy()
            if (stock_df.shape[0] != 252):
                # Not enough records for computation
                print(
                    f'({count}/{len(all_stock_id_list)}) {stock_id} has inadequate records till {date}.'
                )
                count += 1
                CMRA_container.append([stock_id, np.nan])
                pass
            else:
                ret_df = stock_df.merge(right=shibor_df,
                                        how='inner',
                                        left_on='trade_date',
                                        right_on='date')
                ret_df['chg'] = ret_df['pct_chg'] / 100  # Not use the pct form
                ret_df = ret_df.sort_values(by=['trade_date'], ascending=False)
                monthly_excessive_ret_list = []
                for month in range(1, 13):
                    month_ret_df = ret_df.iloc[21 * (month - 1):21 *
                                               month, :].copy()
                    month_ret_df['log_ret'] = np.log(1 + month_ret_df['chg'])
                    month_ret_df['log_risk_free'] = np.log(
                        1 + month_ret_df['daily_on'])
                    month_ret = np.sum(month_ret_df['log_ret'])
                    month_risk_free = np.sum(month_ret_df['log_risk_free'])
                    monthly_excessive_ret_list.append(month_ret -
                                                      month_risk_free)
                Z_list = np.cumsum(monthly_excessive_ret_list)
                CMRA = np.max(Z_list) - np.min(Z_list)
                print(
                    f'({count}/{len(all_stock_id_list)}) CMRA of {stock_id} in {date} is {CMRA}.'
                )
                count += 1
                CMRA_container.append([stock_id, CMRA])
        return_df = pd.DataFrame(CMRA_container,
                                 columns=['ts_code', 'CMRA_raw'])
        return return_df

    def get_daily_residual_volatility(self, date=None):
        RV_HSIGMA_df = self._get_daily_RV_HSIGMA(date)
        RV_CMRA_df = self._get_daily_RV_CMRA(date)
        RV_DASTD_df = self._get_daily_RV_DASTD(date)
        RV_df = RV_HSIGMA_df.merge(right=RV_CMRA_df, on='ts_code', how='outer', validate = '1:1')
        RV_df = RV_df.merge(right=RV_DASTD_df, on='ts_code', how='outer', validate = '1:1')
        return RV_df

    def get_daily_liquidity(self, date=None):
        if date == None:  # Default: Get latest date
            date = self.latest_date
        else:
            date = self._standardize_date(date)

        date_lag = self._get_date_lag(date)
        latest_date_252 = self._get_latest_date(lag=date_lag + 252)
        query_end_date = self._get_query_end_date(lag=date_lag)
        query_all_stock = f'''
        SELECT ts_code, trade_date, turnover_rate_f
        FROM daily
        WHERE trade_date > \'{latest_date_252}\'
        AND trade_date < \'{query_end_date}\';
        '''
        all_stock_df = pd.read_sql_query(query_all_stock,
                                         con=self.alchemy_engine)
        # Get all stock_id
        all_stock_id_list = list(set(all_stock_df['ts_code'].to_list()))
        # Get all stock_id
        liquidity_container = []
        count = 1
        for stock_id in all_stock_id_list:
            stock_df = all_stock_df[all_stock_df['ts_code'] == stock_id].copy()
            if (stock_df.shape[0] != 252):
                # Not enough records for computation
                print(
                    f'({count}/{len(all_stock_id_list)}) {stock_id} has inadequate records till {date}.'
                )
                count += 1
                liquidity_container.append([stock_id, np.nan, np.nan, np.nan])
                pass
            else:
                STOM_list = []
                for month in range(1, 13):
                    month_df = stock_df.iloc[21 * (month - 1):21 *
                                             month, :].copy()
                    STOM = np.log(np.sum(month_df['turnover_rate_f']))
                    STOM_list.append(STOM)
                STOM = STOM_list[0]
                # STOQ
                STOM_3 = STOM_list[:3]
                STOQ = np.log(np.mean(np.exp(STOM_3)))
                # STOA
                STOA = np.log(np.mean(np.exp(STOM_list)))
                print(
                    f'({count}/{len(all_stock_id_list)}) STOM: {STOM}, STOQ: {STOQ}, STOA: {STOA}.'
                )
                count += 1
                liquidity_container.append([stock_id, STOM, STOQ, STOA])
        return_df = pd.DataFrame(liquidity_container,
                                 columns=['ts_code', 'STOM', 'STOQ', 'STOA'])
        return return_df

    def get_daily_leverage(self, date=None):
        if date == None:  # Default: Get latest date
            date = self.latest_date
        else:
            date = self._standardize_date(date)

        query = f'''
            SELECT DISTINCT end_date
            FROM balance;
            '''
        report_date_df = pd.read_sql_query(query, con=self.alchemy_engine)
        report_date_df['last_disclosure_date'] = report_date_df['end_date'].apply(
            lambda x: self._last_disclosure_date(x))
        row_index = np.sum(report_date_df['last_disclosure_date'] < date) - 1
        valid_last_date = report_date_df.iloc[row_index, 0].strftime(
            '%Y-%m-%d')
        print(f'{date}: Use {valid_last_date} for query.')
        balance_query = f'''
            SELECT ts_code,end_date, total_share,total_assets,total_cur_liab,total_liab,oth_eqt_tools_p_shr
            FROM balance
            WHERE end_date = \'{valid_last_date}\';
            '''
        bal_df = pd.read_sql_query(balance_query, con=self.alchemy_engine)

        cap_query = f'''
            SELECT ts_code, total_mv
            FROM daily
            WHERE trade_date = \'{date}\';
            '''
        cap_df = pd.read_sql_query(cap_query, con=self.alchemy_engine)
        cap_df['total_mv_real'] = cap_df['total_mv'] * 10000

        df = bal_df.merge(right=cap_df, how='inner', on='ts_code')
        df = df.fillna(0)  # only affecting total_cur_liab and preferred_stock
        df['long_liab'] = df['total_liab'] - df['total_cur_liab']
        df['MLEV'] = (df['total_mv_real'] + df['oth_eqt_tools_p_shr'] +
                      df['long_liab']) / df['total_mv_real']
        df['DTOA'] = df['total_liab'] / df['total_assets']
        df['BLEV'] = (df['total_share'] + df['oth_eqt_tools_p_shr'] +
                      df['long_liab']) / df['total_share']
        df['leverage_raw'] = 0.38 * df['MLEV'] + \
            0.35 * df['DTOA'] + 0.27 * df['BLEV']

        return_df = df[['ts_code', 'leverage_raw']].copy()
        return return_df

    def get_daily_earnings_yield(self, date=None):
        if date == None:  # Default: Get latest date
            date = self.latest_date
        else:
            date = self._standardize_date(date)
            
        # EPIBS
        est_pe_query = '''
                    SELECT 
                    '''
        

## Test

In [3]:
Loader = DataLoader()

Lastest date(lag = 0 from today): 2021-09-03


In [4]:
RV_df = Loader.get_daily_residual_volatility()

Lastest date(lag = 252 from today): 2020-08-24
(1/4474) Volatility of residual return of 688222.SH in 2021-09-03 is 1.0170663740875647.
(2/4474) Volatility of residual return of 601518.SH in 2021-09-03 is 1.0659603147736503.
(3/4474) Volatility of residual return of 300622.SZ in 2021-09-03 is 1.0807257907949066.
(4/4474) Volatility of residual return of 000031.SZ in 2021-09-03 is 1.084714160662398.
(5/4474) Volatility of residual return of 601899.SH in 2021-09-03 is 0.9486583393187875.
(6/4474) Volatility of residual return of 300623.SZ in 2021-09-03 is 1.0133355728048474.
(7/4474) Volatility of residual return of 300855.SZ in 2021-09-03 is 1.080136431293269.
(8/4474) Volatility of residual return of 003027.SZ in 2021-09-03 is 1.1147509818384187.
(9/4474) Volatility of residual return of 600100.SH in 2021-09-03 is 1.0511489226511408.
(10/4474) Volatility of residual return of 600475.SH in 2021-09-03 is 1.0550851295081616.
(11/4474) 601528.SH has inadequate records till 2021-09-03.
(12/

In [9]:
RV_df.to_csv('../database/data/RV_demo.csv', index=False)

## Scrach Paper

In [7]:
with open('../config/config.json', 'r') as f:
    config = json.load(f)
con = pymysql.connect(**config)
cur = con.cursor()

# sqlalchemy
alchemy_str = 'mysql+pymysql://' + config['user'] + ':' + config[
    'password'] + '@' + config['host'] + ':' + str(
        config['port']) + '/' + config['database']
alchemy_engine = sqlalchemy.create_engine(alchemy_str)

In [8]:
cal_query = '''
            SELECT DISTINCT trade_date
            FROM asset_return;
            '''
cal_df = pd.read_sql_query(cal_query, con=alchemy_engine)
cal_df = cal_df.sort_values(by=['trade_date'], ascending=False)


def _get_latest_date(lag=0):  # Get latest date in database BARRA
    date_dt = cal_df.iloc[lag, 0]
    date = date_dt.strftime('%Y-%m-%d')
    print(f'Lastest date(lag = {lag}): {date}')
    return date